In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import viz

COLORS, PAGE_WIDTH, ROW_HEIGHT = viz.set_plots()


# rolling mean in numpy:


def rolling_mean(data, window_size):
    return np.convolve(data, np.ones(window_size) / window_size, mode="valid")

In [ ]:
prices = pd.read_csv("../data/prices.csv", index_col=0, parse_dates=True)
prices2023 = prices.loc[f"2023-01-01":f"2023-01-31"].lmp

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(PAGE_WIDTH, ROW_HEIGHT))
prices2023.plot(ax=ax)
ax.set_title("2023 LMP")
ax.set_ylabel("LMP ($/MWh)")
plt.show()

In [ ]:
# from dqnRun import main as dqnMain

# dqnMain(nYears=None, nMonths=40, nDays=None, month=[1], test=1)

In [ ]:
losses, rewards = np.load("DQNResults/dqn_losses.npy"), np.load(
    "DQNResults/dqn_rewards.npy"
)
f, ax = plt.subplots(2, 1, figsize=(PAGE_WIDTH, 2 * ROW_HEIGHT))
ax[0].plot(rewards[:], label="Reward")
ax[0].plot(rolling_mean(rewards, 10)[:], label="Rolling Mean")
ax[0].set_title("DQN Training Rewards")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Loss")

ax[1].plot(losses[:], label="Loss")
ax[1].plot(rolling_mean(losses, 10)[:], label="Rolling Mean")
ax[1].set_title("DQN Training Loss")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Loss")

plt.tight_layout()
plt.savefig("DQNResults/dqn_rewards.png")
plt.show()

In [ ]:
history = pd.read_csv("DQNResults/dqn_history.csv", index_col=0, parse_dates=True)

START, END = "2023-01-01", "2023-01-25"

viz.plotOperations(history, START, END, save_path="DQNResults/policy.png")

In [ ]:
from DQNPolicy import DQNPolicy
from tqdm import tqdm

rewards = []
max_power = []
for _ in tqdm(range(1)):
    policy = DQNPolicy(
        [], pricesTest=prices2023, network_dict="DQNResults/dqn_model.pth"
    )
    history = policy.run(prices2023)
    rewards.append(history.reward.sum())
    max_power.append(history.load.max())
    # print(history.reward.sum())

print("Reward Metrics: ", np.mean(rewards), "$")
print("Max Power Metrics: ", np.mean(max_power), "MW")

In [ ]:
START, END = "2023-01-01", "2023-01-25"

viz.plotOperations(history, START, END, save_path="DQNResults/policy.png")